<a href="https://colab.research.google.com/github/nekdil566/Requirements-Issues-Detection-Tool/blob/main/GPT3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://colab.research.google.com/drive/1BHL_jaEVuTy6cfYBk8n-8iYvSeJICo12?usp=sharing#scrollTo=Yc6pgsbUCnLj

In [ ]:
!pip install --upgrade scikit-learn

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import re
import string
import random
from sklearn.model_selection import train_test_split
from collections import Counter
import seaborn as sns
from io import StringIO
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from IPython.display import display
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import top_k_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import mean_squared_error

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#@title Record Each Cell's Execution Time
!pip install ipython-autotime

%load_ext autotime

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import pandas as pd
data = pd.read_csv(", encoding='latin1', usecols=['Base_Reviews','category'])
data = data[pd.notnull(data['category'])]
print(data['Base_Reviews'].astype(str).apply(lambda x: len(x.split(' '))).sum())

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data.info()

In [ ]:
data.head(5)

In [ ]:
data.My_Labels.value_counts()

In [ ]:
data.describe()

In [ ]:
data.Base_Reviews.duplicated(keep="first").value_counts()

In [ ]:
data.drop_duplicates(subset="Base_Reviews",keep="first",inplace=True,ignore_index=True)
data.describe()

In [ ]:
data.My_Labels.value_counts()

In [ ]:
fig = plt.figure(figsize=(8,6))
data.groupby('category').Base_Reviews.count().plot.bar(ylim=0)
plt.show()

In [ ]:
data['words'] = [len(x.split()) for x in data['Base_Reviews'].tolist()]

In [ ]:
data.tail(5)

In [ ]:
data['words'].describe()

In [ ]:
# Distribution of words in each review
data.Base_Reviews.str.split().\
    map(lambda x: len(x)).\
    hist()

In [ ]:
print(max(data['Base_Reviews'], key=len))

In [ ]:
print(min(data['Base_Reviews'], key=len))

In [ ]:
data["Base_Reviews"]

In [ ]:
data["My_Labels"]

In [ ]:
data["category"] = data["category"].astype('category') #By converting an existing Series or column to a category dtype
data.dtypes

In [ ]:
data["category_id"] = data["category"].cat.codes
data.tail()

In [ ]:
data.dtypes

In [ ]:
id_to_category = pd.Series(data.category.values,index=data.category_id).to_dict()
id_to_category

In [ ]:
category_to_id= {v:k for k,v in id_to_category.items()}
category_to_id

In [ ]:
number_of_categories = len(category_to_id)
number_of_categories

In [ ]:
wordList = list()
for i in range(len(data)):
    temp = data.Base_Reviews[i].split()
    for k in temp:
        k = re.sub("[^a-zA-ZğĞüÜşŞıİöÖçÇ]","",k)
        if k != "":
            wordList.append(k)

In [ ]:
wordCount = Counter(wordList)
countedWordDict = dict(wordCount)
sortedWordDict = sorted(countedWordDict.items(),key = lambda x : x[1],reverse=True)

print("Most Used 20 Words")
for word,counted in sortedWordDict[0:20]:
    print("{} : {}".format(word,counted))

In [ ]:
for i in data["Base_Reviews"][7:10]:
    if "oku" in i:
        print(i)
        print("*"*20)

In [ ]:
# Let us take a look at the most frequently used word in the reviews
from wordcloud import WordCloud
word_cloud = WordCloud(width = 1000,
                       height = 800,
                       colormap = 'Blues', 
                       margin = 0,
                       max_words = 183,  
                       max_font_size = 120, min_font_size = 15,  
                       background_color = "white").generate(" ".join(data['Base_Reviews']))

plt.figure(figsize = (10, 15))
plt.imshow(word_cloud, interpolation = "gaussian")
plt.axis("off")
plt.show()

In [ ]:
df2 = data.sample(1000, random_state=1).copy()

In [ ]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,
                        ngram_range=(1,1))

# We transform each review into a vector
df2_features = tfidf.fit_transform(df2.Base_Reviews).toarray()

df2_labels = df2.category_id

print("Each of the %d reviews is represented by %d features (TF-IDF score of unigrams and bigrams)" %(df2_features.shape))

In [ ]:
df2_features.shape

In [ ]:
# Finding the three most correlated terms with each of the categories
N = 20
for category, category_id in sorted(category_to_id.items()):
  features_chi2 = chi2(df2_features, df2_labels == category_id)
  indices = np.argsort(features_chi2[0])
  feature_names = np.array(tfidf.get_feature_names())[indices]
  unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
  bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
  print("\n==> %s:" %(category))
  print("  * Most Correlated Unigrams are: %s" %(', '.join(unigrams[-N:])))
  print("  * Most Correlated Bigrams are: %s" %(', '.join(bigrams[-N:])))

In [ ]:
#limit the number of samples to be used in code runs
data_size= 9870

In [ ]:
vocab_size = 2000  # Only consider the top 90k words
maxlen = 50  # Max sequence size                           # orignale 50 

In [ ]:
# save features and targets from the 'data' (raw data)
features, targets = data['Base_Reviews'], data['category_id']

In [ ]:
train_features, test_features, train_targets, test_targets = train_test_split(
        features, targets,
        train_size=0.8,
        test_size=0.2,
        # random but same for all run, also accurancy depends on the
        # selection of data e.g. if we put 10 then accuracy will be 1.0
        # in this example
        random_state=23,
        # keep same proportion of 'target' in test and target data
        stratify=targets
    )

In [ ]:
def custom_standardization(input_string): 
    first_input = tf.strings.regex_replace(input_string, "İ", "I")
    lowercased = tf.strings.lower(first_input, encoding='utf-8') #Turn to lower case
    stripped_html = tf.strings.regex_replace(lowercased, "<br />", " ") #Remove html line-break tags
    no_numbers = tf.strings.regex_replace(stripped_html, "\w*\d\w*"," ") #Remove numbers
    no_punctuation = tf.strings.regex_replace(no_numbers,'[%s]' % re.escape(string.punctuation),'') #Remove punctuations
    no_turkish_character = tf.strings.regex_replace(no_punctuation, "ç", "c") #Replace turkish characters
    no_turkish_character = tf.strings.regex_replace(no_turkish_character, "ğ", "g")
    no_turkish_character = tf.strings.regex_replace(no_turkish_character, "ı", "i")
    no_turkish_character = tf.strings.regex_replace(no_turkish_character, "ö", "o")
    no_turkish_character = tf.strings.regex_replace(no_turkish_character, "ş", "s")
    no_turkish_character = tf.strings.regex_replace(no_turkish_character, "ü", "u")
    no_read_more = tf.strings.regex_replace(no_turkish_character, "...devamini oku", " ") #Remove "Read More"
    return no_read_more

In [ ]:
# output = custom_standardization('Pijamalı İlginç Otobüs Çıkıp Öldü Gözleri')
# print(output)

In [ ]:
# Create a vectorization layer and adapt it to the text
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size - 1,
    output_mode="int",
    output_sequence_length=maxlen,
)

In [ ]:
vectorize_layer.adapt(train_features)
vocab = vectorize_layer.get_vocabulary() 

In [ ]:
print("vocab has the ", len(vocab)," entries")
print("vocab has the following first 10 entries")
for word in range(10):
  print(word, " represents the word: ", vocab[word])

In [ ]:
# Vector for word "good"
print(vectorize_layer("good"))

In [ ]:
# Vector for word "bu güzel gün"
print(vectorize_layer("Nothing dog about it. It's like a spa video"))

In [ ]:
# Pickle the config and weights
pickle.dump({'config': vectorize_layer.get_config(),
             'weights': vectorize_layer.get_weights()}
            , open("tv_layer.pkl", "wb"))

# Later you can unpickle and use 
# `config` to create object and 
# `weights` to load the trained weights. 

In [ ]:
from_disk = pickle.load(open("tv_layer.pkl", "rb"))
new_vectorize_layer = TextVectorization.from_config(from_disk['config'])
# You have to call `adapt` with some dummy data (BUG in Keras)
new_vectorize_layer.adapt(tf.data.Dataset.from_tensor_slices(["xyz"]))
new_vectorize_layer.set_weights(from_disk['weights'])

In [ ]:
# Vector for word "bu"
print(new_vectorize_layer("good"))

In [ ]:
# Vector for word "bu güzel gün"
print(new_vectorize_layer("Nothing dog about it. It's like a spa video"))

In [ ]:
# Vector for word "bu güzel gün"
print(new_vectorize_layer("Nothing dog about it. It's like a spa video"))


In [ ]:
def causal_attention_mask(batch_size, n_dest, n_src, dtype):
    """
    Mask the upper half of the dot product matrix in self attention.
    This prevents flow of information from future tokens to current token.
    1's in the lower triangle, counting from the lower right corner.
    """
    i = tf.range(n_dest)[:, None]
    j = tf.range(n_src)
    m = i >= j - n_src + n_dest
    mask = tf.cast(m, dtype)
    mask = tf.reshape(mask, [1, n_dest, n_src])
    mult = tf.concat(
        [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0
    )
    return tf.tile(mask, mult)


class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1, **kwargs):
        super(TransformerBlock, self).__init__()
        self.embed_dim=embed_dim
        self.num_heads = num_heads
        self.ff_dim =ff_dim
        self.att = layers.MultiHeadAttention(num_heads, embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        # for masked-self attention add the mask:
        # causal_mask = causal_attention_mask(batch_size, seq_len, seq_len, tf.bool)
        # attention_output = self.att(inputs, inputs,attention_mask=causal_mask)
        
        attention_output = self.att(inputs, inputs) 
        
        attention_output = self.dropout1(attention_output)
        out1 = self.layernorm1(inputs + attention_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)
    
    # https://newbedev.com/saving-keras-models-with-custom-layers
    def get_config(self):
        config = super(TransformerBlock, self).get_config()
        config.update({
            'att': self.att,
            'ffn': self.ffn,
            'layernorm1': self.layernorm1,
            'layernorm2':self.layernorm2,
            'dropout1':self.dropout1,
            'dropout2':self.dropout2,
            'embed_dim': self.embed_dim,
            'num_heads':self.num_heads,
            'ff_dim':self.ff_dim 

        })
        return config
        #tf.keras.models.save_model(model, 'model.h5')
        #new_model = tf.keras.models.load_model('model.h5', custom_objects={'CustomLayer': CustomLayer})

In [ ]:
class TokenPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size,  embed_dim, **kwargs):
        super(TokenPositionEmbedding, self).__init__()
        self.maxlen = maxlen
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, X):
        maxlen = tf.shape(X)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        X = self.token_emb(X)
        return X + positions 
        
    # https://newbedev.com/saving-keras-models-with-custom-layers
    def get_config(self):
        config = super(TokenPositionEmbedding, self).get_config()
        config.update({
            'token_emb': self.token_emb,
            'pos_emb': self.pos_emb,
            'maxlen': self.maxlen,
            'vocab_size': self.vocab_size,
            'embed_dim': self.embed_dim
        })
        return config
        #tf.keras.models.save_model(model, 'model.h5')
        #new_model = tf.keras.models.load_model('model.h5', custom_objects={'CustomLayer': CustomLayer})
        

In [ ]:
embed_dim = 256  # Embedding size for each token
num_heads = 2  # Number of attention heads
feed_forward_dim = 256  # Hidden layer size in feed forward network inside transformer


def create_model():
    inputs_tokens = layers.Input(shape=(maxlen,), dtype=tf.int32)
    embedding_layer = TokenPositionEmbedding(maxlen, vocab_size, embed_dim)
    x = embedding_layer(inputs_tokens)
    transformer_block1 = TransformerBlock(embed_dim, num_heads, feed_forward_dim)
    transformer_block2 = TransformerBlock(embed_dim, num_heads, feed_forward_dim)
    x = transformer_block1(x)
    x = transformer_block2(x)
    x = layers.Flatten()(x)
    outputs = layers.Dense(number_of_categories)(x)
    model = keras.Model(inputs=inputs_tokens, outputs=outputs)
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metric_fn  = tf.keras.metrics.SparseCategoricalAccuracy()
    model.compile(optimizer="adam", loss=loss_fn, metrics=metric_fn)  
    
    return model
my_model=create_model()

In [ ]:
my_model.summary()

In [ ]:
tf.keras.utils.plot_model(my_model,show_shapes=True)

In [ ]:
checkpoint_filepath = './checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False)

In [ ]:
X,y = train_features,train_targets

balanced_accuracy_scores = []
matthews_corrcoef_scores = []
f1_scores = []
conf_matrix_list_of_arrays = []

# prepare cross validation
n=10
seed=1
k_fold = KFold(n_splits=n, random_state=seed, shuffle=True)

for train_index, test_index in k_fold.split(X,y):
  X_train, X_test = X.iloc[train_index], X.iloc[test_index]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]

  X_train_dtm = vectorize_layer(X_train)
  X_test_dtm = vectorize_layer(X_test)
  history = my_model.fit(X_train_dtm, y_train, verbose=1, epochs=1, callbacks=[model_checkpoint_callback])
  y_pred_class = my_model.predict(X_test_dtm)

  y_pred_class = np.argmax(y_pred_class,axis=1)
  
  conf_matrix = confusion_matrix(y_test, y_pred_class)
  conf_matrix_list_of_arrays.append(conf_matrix)
  
  balanced_accuracy_scores.append(balanced_accuracy_score(y_test, y_pred_class))
  matthews_corrcoef_scores.append(matthews_corrcoef(y_test, y_pred_class))
  f1_scores.append(f1_score(y_test, y_pred_class, average='weighted'))

In [ ]:
mean_of_conf_matrix_arrays = np.mean(conf_matrix_list_of_arrays, axis=0)
print('Mean of conf_matrix: ', mean_of_conf_matrix_arrays)

In [ ]:
conf_mat = mean_of_conf_matrix_arrays.astype(int)

In [ ]:
category_id_df = data[['category', 'category_id']].drop_duplicates()

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sns.heatmap(conf_mat, annot=True, cmap="Blues", fmt='d',
            xticklabels=category_id_df.category.values, 
            yticklabels=category_id_df.category.values)
plt.ylabel('Actual', fontsize=12)
plt.xlabel('Predicted', fontsize=12)
plt.title("CONFUSION MATRIX \n", size=16);

In [ ]:
balanced_accuracy_scores = np.array(balanced_accuracy_scores)
print('Mean of balanced_accuracy_scores: ', np.mean(balanced_accuracy_scores, axis=0))

In [ ]:
matthews_corrcoef_scores = np.array(matthews_corrcoef_scores)
print('Mean of matthews_corrcoef_scores: ', np.mean(matthews_corrcoef_scores, axis=0))

In [ ]:
f1_scores = np.array(f1_scores)
print('Mean of f1_scores: ', np.mean(f1_scores, axis=0))

In [ ]:
test_features_vc = vectorize_layer(test_features)
y_pred_test = my_model.predict(test_features_vc)
y_pred_test = np.argmax(y_pred_test,axis=1)
 
balanced_accuracy_score_test = balanced_accuracy_score(test_targets, y_pred_test)
print('balanced_accuracy_score_test :',balanced_accuracy_score_test)
matthews_corrcoef_score_test = matthews_corrcoef(test_targets, y_pred_test)
print('matthews_corrcoef_score_test :',matthews_corrcoef_score_test)
f1_score_test = f1_score(test_targets, y_pred_test, average='weighted')
print('f1_score_test :',f1_score_test)


In [ ]:
new_review = "I removed it."
predictions=my_model.predict(vectorize_layer([new_review]))
for pred in predictions:
  print(id_to_category[np.argmax(pred)])

In [ ]:
examples = [
  "I removed it.",
  "Does what it says great app!",
  "Not compatable.",
  "Ok.",
  "don't waste ur time , this app won't help ur Amazon Tablet run smoother or faster.  skip it",
]
predictions=my_model.predict(vectorize_layer(examples))
for pred in predictions:
  print(id_to_category[np.argmax(pred)])

In [ ]:
tf.keras.models.save_model(my_model, 'MultiClassTextClassifier')